In [14]:
import pandas as pd

jsonn = pd.read_json("financial_ratios.jsonl", lines = True)


jsonn.to_csv("financial_ratios.csv", index=False)

In [ ]:
import pandas as pd

geo_data = pd.read_xml("geographic_data.xml")

geo_data.to_csv("geographic_data.csv", index=False)




# Data Cleaning Start

In [25]:
# starting with demographics file
import pandas as pd
from rapidfuzz import process, fuzz




demo_graphs = pd.read_csv("demographics.csv")
# replace '$' or ',' in annual income column and convert to int

demo_graphs['annual_income'] = (
    demo_graphs['annual_income'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(int)
)

# using Fuzzy matching format values in employement_type column like FT, PT, FULL_TIME, FullTime to the same format 'Full Time'

# Define final categories
CATEGORIES = ["Full Time", "Part Time", "Contractor", "Self Employed"]

# Add explicit short-code mapping
SHORT_MAP = {
    "ft": "Full Time",
    "pt": "Part Time"
}

# Normalize raw text
def clean_text(v):
    if pd.isna(v):
        return ""
    return str(v).lower().replace(" ", "").replace("-", "").replace("_", "")

# Fuzzy match function
def fuzzy_employment(value):
    text = clean_text(value)
    
    if text == "":
        return None

    # Check short-code mapping first
    if text in SHORT_MAP:
        return SHORT_MAP[text]

    # Fuzzy match against categories
    match, score, _ = process.extractOne(
        text,
        CATEGORIES,
        scorer=fuzz.partial_ratio
    )

    if score <= 70:
        return None  # or return value

    return match

# Apply cleaning
demo_graphs["employment_type"] = demo_graphs["employment_type"].apply(fuzzy_employment)

# Save clean DataFrame



demo_graphs.to_csv("demographics.csv", index=False)

demo_graphs

# 

,cust_id,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents
0,10000,41,61800,2.2,Full Time,Graduate,Married,2
1,10001,38,28600,7.0,Full Time,High School,Married,0
2,10002,18,20700,0.8,Full Time,Bachelor,Single,0
3,10003,27,31400,4.8,Full Time,Bachelor,Single,0
4,10004,26,24600,5.2,Full Time,High School,Single,0
...,...,...,...,...,...,...,...,...
89994,99994,52,72300,3.7,Full Time,Bachelor,Single,1
89995,99995,61,20000,4.2,Full Time,High School,Divorced,1
89996,99996,19,20000,1.5,Full Time,High School,Married,3
89997,99997,21,20000,3.5,Full Time,High School,Married,1


In [19]:
# Moving on to the Financial Rations


fin_rat = pd.read_csv("financial_ratios.csv")



# we have to convert all other columns to float after removing some unnecessary symbols like '$' and ','

for col in fin_rat.columns:
    if col == "cust_num":
        pass    
    else:
        fin_rat[col] = fin_rat[col].astype(str).replace(r'[\$,]', '', regex=True).astype(float)


fin_rat.to_csv("financial_ratios.csv", index=False)


fin_rat


,cust_num,monthly_income,existing_monthly_debt,monthly_payment,debt_to_income_ratio,debt_service_ratio,payment_to_income_ratio,credit_utilization,revolving_balance,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow
0,10000,5150.00,738.64,592.13,0.258,0.258402,0.115,0.841,142213.1,142213.1,26886.9,1330.77,15969.24,0.286408,159913.1,3819.230000
1,10001,2383.33,392.21,1013.86,0.590,0.589959,0.425,0.971,75932.2,75932.2,2267.8,1406.07,16872.84,3.986014,189932.2,977.260000
2,10002,1725.00,204.07,317.81,0.303,0.302539,0.184,0.539,22314.6,22314.6,19085.4,521.88,6262.56,0.449275,31614.6,1203.120000
3,10003,2616.67,288.71,234.52,0.200,0.199961,0.090,0.147,8820.0,8820.0,51180.0,523.23,6278.76,0.277070,17520.0,2093.436667
4,10004,2050.00,248.77,334.81,0.285,0.284673,0.163,0.488,24253.6,24253.6,25446.4,583.58,7002.96,0.292683,31453.6,1466.420000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89994,99994,6025.00,2120.97,519.87,0.438,0.438314,0.086,0.359,48680.4,48680.4,86919.6,2640.84,31690.08,0.156293,59980.4,3384.160000
89995,99995,1666.67,138.97,545.44,0.411,0.410646,0.327,0.355,15833.0,15833.0,28767.0,684.41,8212.92,0.825000,32333.0,982.260000
89996,99996,1666.67,129.90,616.96,0.448,0.448116,0.370,0.669,12242.7,12242.7,6057.3,746.86,8962.32,0.890000,30042.7,919.806667
89997,99997,1666.67,162.11,351.00,0.308,0.307866,0.211,0.562,30516.6,30516.6,23783.4,513.11,6157.32,0.585000,42216.6,1153.560000


In [20]:
# Moving to Loan Details

import pandas as pd
from rapidfuzz import process, fuzz

loan_det = pd.read_excel("loan_details.xlsx", "Sheet1")

# converting loan_amount column to integer after removing some unnecessary symbols like '$' and ','

loan_det["loan_amount"] = loan_det["loan_amount"].astype(str).replace(r'[\$,]', '', regex=True).astype(int)



# Read your Excel file

# Final standardized categories
FINAL_CATEGORIES = ["Credit Card", "Home Loan", "Mortgage", "Personal Loan"]

# Short-code mapping for very short inputs
SHORT_MAP = {
    "cc": "Credit Card",
    "hl": "Home Loan",
    "pl": "Personal Loan"
}

# Normalize text
def clean_text(v):
    if pd.isna(v):
        return ""
    return str(v).lower().replace(" ", "").replace("-", "").replace("_", "")

# Fuzzy match function
def fuzzy_loan_type(value):
    text = clean_text(value)
    
    if text == "":
        return None

    # Check short-code mapping first
    if text in SHORT_MAP:
        return SHORT_MAP[text]

    # Fuzzy match against final categories
    match, score, _ = process.extractOne(
        text,
        FINAL_CATEGORIES,
        scorer=fuzz.partial_ratio
    )

    # Only accept good matches
    if score < 70:
        return None

    return match

# Apply cleaning
loan_det['loan_type'] = loan_det['loan_type'].apply(fuzzy_loan_type)

# Save cleaned DataFrame
loan_det.to_csv("loan_details.csv", index=False)

loan_det





,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,Personal Loan,17700,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W
1,10001,Mortgage,114000,180,6.83,Refinance,0.774,Branch,1011,B
2,10002,Personal Loan,9300,36,13.99,Major Purchase,0.000,Online,1084,K
3,10003,Personal Loan,8700,48,13.26,Medical,0.000,Online,1048,A
4,10004,Personal Loan,7200,24,10.77,Debt Consolidation,0.000,Branch,1055,S
...,...,...,...,...,...,...,...,...,...,...
89994,99994,Personal Loan,11300,24,9.70,Home Improvement,0.000,Branch,1050,O
89995,99995,Personal Loan,16500,36,11.67,Other,0.000,Branch,1071,D
89996,99996,Personal Loan,17800,36,14.99,Major Purchase,0.000,Online,1046,J
89997,99997,Credit Card,11700,0,19.10,Revolving Credit,0.000,Branch,997,K


In [21]:
# Moving to Credit History

cred_hist = pd.read_csv("credit_history.csv")

# acc_age_month is rounded up to 1 decimal places as they have only 9999999 or 0000001s in the end
cred_hist["oldest_account_age_months"] = cred_hist["oldest_account_age_months"].astype(float).round(1)

cred_hist.to_csv("credit_history.csv")

cred_hist

,customer_number,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,recent_inquiry_count,num_public_records,num_collections,account_diversity_index
0,10000,696,14,22.8,273.6,169100.0,0.0,2,2,1,0,0.499
1,10001,659,13,3.5,42.0,78200.0,0.0,6,6,0,0,0.298
2,10002,662,3,0.0,0.0,41400.0,0.0,2,2,0,0,0.174
3,10003,676,8,9.0,108.0,60000.0,0.0,1,1,0,0,0.263
4,10004,678,7,8.0,96.0,49700.0,0.0,1,1,0,0,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...
89994,99994,817,10,8.2,98.4,135600.0,0.0,1,1,0,0,0.285
89995,99995,745,9,8.8,105.6,44600.0,0.0,1,1,0,0,0.353
89996,99996,607,11,1.0,12.0,18300.0,0.0,2,2,1,0,0.238
89997,99997,678,10,3.0,36.0,54300.0,0.0,3,3,0,0,0.227


In [9]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

# ============================================================
# 1) SQL CONNECTION (Windows Authentication ✔)
# ============================================================

connection_url = URL.create(
    "mssql+pyodbc",
    username=None,      # Windows Auth → username/password not used
    password=None,
    host="NODIRBEK",
    database="coding_challenge",
    query={"driver": "ODBC Driver 17 for SQL Server", "trusted_connection": "yes"}
)

engine = create_engine(connection_url, fast_executemany=True)

# ============================================================
# 2) FILE PATHS (replace to your actual paths)
# ============================================================

files = {
    "coding_challenge.landing.application_metadata_clean": "application_metadata.csv",
    "coding_challenge.landing.demographics_clean": "demographics.csv",
    "coding_challenge.landing.financial_ratios_clean": "financial_ratios.csv",
    "coding_challenge.landing.loan_details_clean": "loan_details.csv",
    "coding_challenge.landing.credit_history_clean": "credit_history.csv",
    "coding_challenge.landing.geographic_data_clean": "geographic_data.csv"
}

# ============================================================
# 3) LOAD ALL CSVs INTO LANDING TABLES
# ============================================================

for table, file_path in files.items():
    print(f"Loading {file_path} → {table} ...")

    # read CSV
    df = pd.read_csv(file_path)

    # write to SQL (replace data)
    df.to_sql(
        name=table.split('.')[-1],   # table name only
        schema=table.split('.')[1],  # schema name
        con=engine,
        if_exists="replace",         # load fresh
        index=False
    )

    print(f"Finished loading {table}")

print("\nAll landing tables loaded successfully!")


Loading application_metadata.csv → coding_challenge.landing.application_metadata_clean ...
Finished loading coding_challenge.landing.application_metadata_clean
Loading demographics.csv → coding_challenge.landing.demographics_clean ...
Finished loading coding_challenge.landing.demographics_clean
Loading financial_ratios.csv → coding_challenge.landing.financial_ratios_clean ...
Finished loading coding_challenge.landing.financial_ratios_clean
Loading loan_details.csv → coding_challenge.landing.loan_details_clean ...
Finished loading coding_challenge.landing.loan_details_clean
Loading credit_history.csv → coding_challenge.landing.credit_history_clean ...
Finished loading coding_challenge.landing.credit_history_clean
Loading geographic_data.csv → coding_challenge.landing.geographic_data_clean ...
Finished loading coding_challenge.landing.geographic_data_clean

All landing tables loaded successfully!


In [5]:
pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 3.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.0 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.0 MB 5.3 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 5.6 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/11.0 MB 5.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.0 MB 5.8 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 5.0 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 4.8 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 4.9 MB/s eta 0:00:01
   ----------------------------------- ---- 9.7/11.0 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 4.7 MB/s eta 0:00:00
   -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install pandas sqlalchemy pyodbc 


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 2.8 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 2.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 2.3 MB/s eta 0:00:00

   -------------------- ------------------- 2/4 [greenlet]
   -------------------- ------------------- 2/4 [greenlet]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ---------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
